In [2]:
# imports
import openai
import dotenv
import os

from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import load_tools, AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI

In [3]:
# load environment variables
dotenv.load_dotenv()

True

In [8]:
%%timeit
llm = OpenAI(openai_api_key = os.getenv("OPENAI_API_KEY"),
            temperature=1.0)
response = llm("Recommend me a recipe that uses potato and rosemary.")

10.3 s ± 1.88 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
response

'\n\nRoasted Rosemary Potatoes\n\nIngredients:\n\n- 4 large potatoes, cut into wedges\n- 2 tablespoons olive oil\n- 2 teaspoons minced garlic\n- 1 teaspoon minced fresh rosemary\n- Salt and pepper to taste\n\nInstructions:\n\n1. Preheat oven to 400 degrees F (200 degrees C).\n\n2. Place the potato wedges in a large bowl. Add the olive oil, garlic, rosemary, salt and pepper and gently toss to combine.\n\n3. Spread the potatoes out on a baking sheet.\n\n4. Bake for 25 to 30 minutes, or until golden brown and tender. Serve warm.'

In [11]:
# using simple chain (prompt template + llm run)
# define llm
llm = OpenAI(openai_api_key = os.getenv("OPENAI_API_KEY"),
            temperature=1.0)
# define prompt template
prompt_template = PromptTemplate(input_variables=['ingredients'],
                                template="Recommend me a recipe that uses the following ingredients: {ingredients}")
# build chain
chain = LLMChain(llm=llm,prompt=prompt_template,verbose=True)

In [12]:
chain.run("potato, rosemary")



> Entering new LLMChain chain...
Prompt after formatting:
Recommend me a recipe that uses the following ingredients: potato, rosemary

> Finished chain.


'\n\nParmesan Rosemary Baked Potato Wedges. \n\nIngredients\n- 2 large potatoes, washed and cut into wedges\n- 2 tablespoons olive oil\n- 2 cloves garlic, minced\n- 2 teaspoons fresh rosemary, finely chopped\n- 1/4 teaspoon salt\n- Black pepper, to taste\n- 2 tablespoons freshly grated Parmesan cheese\n\nInstructions\n1. Preheat oven to 400°F (205°C).\n2. Lay the potato wedges on a lined baking sheet. \n3. Drizzle with olive oil and toss to coat.  \n4. Sprinkle the garlic, rosemary, salt, and pepper over the potatoes and mix until they are all evenly seasoned. \n5. Bake for 15 minutes and then rotate the baking sheet 180°.\n5. Continue baking for 15 minutes or until potatoes are golden brown and crispy. \n6. Sprinkle with Parmesan cheese and bake for another 1-2 minutes, or until Parmesan has melted. \n7. Serve warm!'

In [4]:
# 3 components: tool, llm, agent
# llm
llm = OpenAI(openai_api_key = os.getenv("OPENAI_API_KEY"),
            temperature=1.0)
# tool
tools = load_tools(tool_names=['serpapi'], llm=llm)
# initialize agent with tools and agent type zero shot
agent = initialize_agent(tools=tools,llm=llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

In [5]:
# run test prompt
# hahahaha sunny anderson
prompt = "What was the weather like in Berlin yesterday? Recommend me a recipe that is appropriate for the weather."
agent.run(prompt)



> Entering new AgentExecutor chain...
 I need to find out what the weather was like in Berlin yesterday.
Action: Search
Action Input: Berlin weather yesterday
Observation: 61 °F · Sunny. 6 mph ...
Thought: I need to find a recipe based on the information I gathered. 
Action: Search
Action Input: Sunny recipe
Observation: https://www.foodnetwork.com/profiles/talent/sunny-anderson/recipes
Thought: I now know the final answer
Final Answer: A recipe from Sunny Anderson that is appropriate for the weather in Berlin yesterday would be Cashew Chicken Pineapple Curry.

> Finished chain.


'A recipe from Sunny Anderson that is appropriate for the weather in Berlin yesterday would be Cashew Chicken Pineapple Curry.'

In [7]:
prompt = "What was the weather like in Berlin yesterday? Recommend me a recipe that is weather appropriate."
agent.run(prompt)



> Entering new AgentExecutor chain...
 I should start by finding out what the weather was like yesterday in Berlin.
Action: Search
Action Input: Weather in Berlin yesterday
Observation: https://www.timeanddate.com/weather/germany/berlin/historic
Thought: The weather was warm and sunny, so a summer recipe would be great.
Action: Search
Action Input: Summer recipes
Observation: Grilled Chicken Breasts With Tadka-ish Sauce · Easiest-Ever Grilled Veggie Burgers · Classic Potato Salad · Raspberry-Cardamom Yogurt Semifreddo.
Thought: I now know what type of recipe to make.
Final Answer: A summer recipe, such as Grilled Chicken Breasts with Tadka-ish Sauce, Easiest-Ever Grilled Veggie Burgers, Classic Potato Salad, or Raspberry-Cardamom Yogurt Semifreddo.

> Finished chain.


'A summer recipe, such as Grilled Chicken Breasts with Tadka-ish Sauce, Easiest-Ever Grilled Veggie Burgers, Classic Potato Salad, or Raspberry-Cardamom Yogurt Semifreddo.'

In [3]:
from langchain.agents.load_tools import get_all_tool_names
get_all_tool_names()

['python_repl',
 'requests',
 'requests_get',
 'requests_post',
 'requests_patch',
 'requests_put',
 'requests_delete',
 'terminal',
 'sleep',
 'wolfram-alpha',
 'google-search',
 'google-search-results-json',
 'searx-search-results-json',
 'bing-search',
 'metaphor-search',
 'ddg-search',
 'google-serper',
 'google-serper-results-json',
 'serpapi',
 'twilio',
 'searx-search',
 'wikipedia',
 'arxiv',
 'pupmed',
 'human',
 'awslambda',
 'sceneXplain',
 'graphql',
 'openweathermap-api',
 'news-api',
 'tmdb-api',
 'podcast-api',
 'pal-math',
 'pal-colored-objects',
 'llm-math',
 'open-meteo-api']

In [3]:
# testing with google search instead of serpapi
# 3 components: tool, llm, agent
# llm
llm = OpenAI(openai_api_key = os.getenv("OPENAI_API_KEY"),
            temperature=1.0)
# tool
tools = load_tools(tool_names=['google-search-results-json'], llm=llm)
# initialize agent with tools and agent type zero shot
agent = initialize_agent(tools=tools,llm=llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)


In [4]:
recipe_name = "avocado toast"
prompt = f"Return maximum 5 recipe urls for the following recipe: {recipe_name}"
agent.run(prompt)



> Entering new AgentExecutor chain...
 I need to find cooking recipes
Action: Google Search Results JSON
Action Input: "avocado toast recipe"
Observation: [{'title': "The Last Avocado Toast Recipe You'll Ever Need", 'link': 'https://www.eatingwell.com/recipe/8029771/avocado-toast-recipe/', 'snippet': 'Mar 21, 2023 ... Top toasted bread with mashed avocado mixture. Drizzle with olive oil and sprinkle over desired toppings.'}, {'title': 'Avocado Toast Recipe (Plus Tips & Variations) - Cookie and Kate', 'link': 'https://cookieandkate.com/avocado-toast-recipe/', 'snippet': 'Toast your slice of bread until golden and firm. · Remove the pit from your avocado. Use a big spoon to scoop out the flesh. · Spread avocado on top of your toast\xa0...'}, {'title': 'Avocado Toast - The Forked Spoon', 'link': 'https://theforkedspoon.com/avocado-toast/', 'snippet': 'Feb 28, 2022 ... 2 slices bread - toasted (multi-grain, sourdough, wheat, gluten-free, etc.) ; 1 large avocado ; 1 teaspoon fresh lemon j

"1. The Last Avocado Toast Recipe You'll Ever Need - https://www.eatingwell.com/recipe/8029771/avocado-toast-recipe/\n2. Avocado Toast Recipe (Plus Tips & Variations) - Cookie and Kate - https://cookieandkate.com/avocado-toast-recipe/\n3. Avocado Toast - The Forked Spoon - https://theforkedspoon.com/avocado-toast/\n4. The Best Avocado Toast Recipe | Food Network Kitchen | Food ... - https://www.foodnetwork.com/recipes/food-network-kitchen/avocado-toasts-3364588"

In [5]:
recipe_name = "Braised Pork Belly (红烧肉, hóng shāo ròu)"
prompt = f"Return maximum 5 recipe urls for the following recipe: {recipe_name}"
agent.run(prompt)



> Entering new AgentExecutor chain...
 I should search for recipes that include the ingredients in this specific dish 
Action: Google Search Results JSON
Action Input: "Braised Pork Belly recipe"
Observation: [{'title': 'Shanghai-Style Braised Pork Belly (Hong Shao Rou) - The Woks of ...', 'link': 'https://thewoksoflife.com/shanghai-style-braised-pork-belly/', 'snippet': 'Apr 14, 2014 ... Cover and simmer over medium heat for about 45 minutes to 1 hour until pork is fork tender. Every 5-10 minutes, stir to prevent burning and add\xa0...'}, {'title': 'Braised Pork Belly (Hong Shao Rou/红烧肉), an Easy Version - Red ...', 'link': 'https://redhousespice.com/red-cooked-pork-belly/', 'snippet': 'Jan 29, 2021 ... Bring the liquid to a full boil. Cover with a lid and leave to simmer over low heat for 1-1.5 hour until the meat is fork-tender (During the\xa0...'}, {'title': 'Braised Pork Belly with Pickled Radishes Recipe - Gail Simmons', 'link': 'https://www.foodandwine.com/recipes/braised-pork

'The maximum 5 recipe urls for the following recipe: Braised Pork Belly (红烧肉, hóng shāo ròu) are https://thewoksoflife.com/shanghai-style-braised-pork-belly/, https://redhousespice.com/red-cooked-pork-belly/, https://www.foodandwine.com/recipes/braised-pork-belly-pickled-radishes, https://www.tasteasianfood.com/braised-pork/, and https://www.seriouseats.com/recipes/2010/12/sous-vide-braised-pork-belly-recipe.html.'

In [6]:
recipe_name = "Lok lak (ល្ហុក្ខិតខ្មែរ, Lohk lahk)"
prompt = f"Return maximum 5 recipe urls for the following recipe: {recipe_name}"
agent.run(prompt)



> Entering new AgentExecutor chain...
 There are many results when searching for this recipe--I should narrow it down.
Action: Google Search Results JSON
Action Input: "Lok lak recipe"
Observation: [{'title': 'Lok Lak (Shaking Beef) Recipe', 'link': 'https://www.simplyrecipes.com/lok-lak-shaking-beef-recipe-7089865', 'snippet': 'Feb 13, 2023 ... Ingredients · 2 tablespoons oyster sauce · 1 tablespoon tomato paste · 2 tablespoons fish sauce · 1/2 tablespoon dark brown sugar · 2 teaspoons\xa0...'}, {'title': "Cambodian Pepper Beef (Lok Lak) | Marion's Kitchen", 'link': 'https://www.marionskitchen.com/cambodian-pepper-beef-lok-lak/', 'snippet': 'Marinade: 1 tbsp oyster sauce. 1 tbsp soy sauce. 2 tsp sweet dark soy sauce. 1 tsp ground Kampot pepper*. 1 tsp cornflour (cornstarch). ½ tsp baking soda · Stir-\xa0...'}, {'title': 'Cambodian Black Pepper Beef (Lok Lak) Recipe - Nite Yun', 'link': 'https://www.foodandwine.com/recipes/cambodian-black-pepper-beef-lok-lak', 'snippet': 'Jul 1, 2019

'1. https://www.simplyrecipes.com/lok-lak-shaking-beef-recipe-7089865\n2. https://www.marionskitchen.com/cambodian-pepper-beef-lok-lak/\n3. https://www.foodandwine.com/recipes/cambodian-black-pepper-beef-lok-lak\n4. https://www.food.com/recipe/beef-lok-lak-cambodian-recipe-496461\n5. https://www.simplyrecipes.com/lany-phlong-5509327'

Chained steps
- user input
- recipe reco (allow selection so divide result by button)
- recipe search on web
- share back url of top web results

In [7]:
recipe_name = "Lok lak (ល្ហុក្ខិតខ្មែរ, Lohk lahk)"
context = "beginner cambodian dish"
prompt = f"Return maximum 5 recipe urls for the following recipe in the context of {context}: {recipe_name}"
agent.run(prompt)



> Entering new AgentExecutor chain...
 I need to use a tool to gather relevant search results
Action: Google Search Results JSON
Action Input: "Lok lak beginner cambodian recipes"
Observation: [{'title': "Cambodian Pepper Beef (Lok Lak) | Marion's Kitchen", 'link': 'https://www.marionskitchen.com/cambodian-pepper-beef-lok-lak/', 'snippet': "It's one of my favourite ways to eat sirloin steak – and it's ready in under 30 minutes. Winner! WATCH THIS RECIPE. Cambodian Pepper Beef (Lok Lak)."}, {'title': 'Lok Lak (Shaking Beef) Recipe', 'link': 'https://www.simplyrecipes.com/lok-lak-shaking-beef-recipe-7089865', 'snippet': "Feb 13, 2023 ... Lok lak is a Cambodian beef dish served with crispy lettuce and fresh ... Lok lak is for a special occasion, but it's also easy enough for a\xa0..."}, {'title': 'Beef Lok Lak (Cambodian Recipe) Recipe - Food.com', 'link': 'https://www.food.com/recipe/beef-lok-lak-cambodian-recipe-496461', 'snippet': 'ingredients · 300 g beef steaks, sliced (best you ca

'Agent stopped due to iteration limit or time limit.'

In [8]:
recipe_name = "fried rice"
context = "beginner cambodian dish"
prompt = f"Return maximum 5 recipe urls for the following recipe in the context of {context}: {recipe_name}"
agent.run(prompt)



> Entering new AgentExecutor chain...
 Let's search for recipes that are beginner-friendly
Action: Google Search Results JSON
Action Input: beginner cambodian dish: fried rice recipes
Observation: [{'title': 'Cambodian Fried Rice Recipe – How to Make the Best Bai Cha', 'link': 'https://grantourismotravels.com/cambodian-fried-rice-recipe-bai-cha/', 'snippet': 'Ingredients · 150 g Siem Reap sausages quartered lengthways and sliced · 4 eggs beaten · 80 g carrot diced finely · 80 g french beans diced finely · 3 garlic cloves\xa0...'}, {'title': 'Easy Egg Fried Rice Recipe - Cambodia Recipe', 'link': 'https://cambodiarecipe.com/recipe/easy-egg-fried-rice-recipe/', 'snippet': 'Jan 6, 2023 ... Directions to make Easy Egg Fried Rice recipe · In a hot wok on medium heat add 3 T of vegetables oil till hot · Add garlic stir fry till brown\xa0...'}, {'title': 'Chinese Sausage Fried Rice (Lop Cheung Chow Fan): A 20-Minute ...', 'link': 'https://thewoksoflife.com/chinese-sausage-fried-rice/', 'sni

'5 beginner-friendly Cambodian fried rice recipes, including Chinese Sausage Fried Rice (Lap Cheong Fried Rice), Pineapple Fried Rice (Thai), Easy Fried Rice, and Basil Fried Rice.'

### Formatting test for output on app

In [3]:
sample_response_text = """
{
   "results":[
      {
         "title":"Red Beans and Rice",
         "cuisine":"Louisiana Creole",
         "description":"A spicy Cajun dish made with red kidney beans, Andouille sausage, and served over rice. It is a staple dish in New Orleans and is traditionally served on Mondays.",
         "pronunciation":"<loo-ee-zi-ah-nuh kree-ohl>",
         "original_title":"Riz Jaune et Haricots Rouge"
      },
      {
         "title":"Frijoles Charros",
         "cuisine":"Mexican",
         "description":"A hearty Mexican dish made with kidney beans, bacon, chorizo, and served with a side of corn or flour tortillas. It is a popular dish served at family gatherings and celebrations.",
         "pronunciation":"<free-hoh-lays charr-ohs>",
         "original_title":"Frijoles Charros"
      },
      {
         "title":"Greek Kidney Bean Salad",
         "cuisine":"Greek",
         "description":"A light and refreshing Greek salad made with kidney beans, feta cheese, and a lemon and olive oil dressing. It is a great side dish for any Greek meal or as a delicious vegetarian main course.",
         "pronunciation":"<greek ki-d-nee been sah-lahd>",
         "original_title":"Σαλάτα Φασολάκια Κοκκινιστά"
      },
      {
         "title":"Indian Rajma Masala",
         "cuisine":"Indian",
         "description":"A popular North Indian dish made with kidney beans in a spicy tomato-based sauce. It is often served with steamed rice or naan bread and is a staple dish in many Indian households.",
         "pronunciation":"<in-dee-an rahj-mah mah-sah-lah>",
         "original_title":"राजमा मसाला"
      },
      {
         "title":"Classic Three Bean Salad",
         "cuisine":"no specific cuisine",
         "description":"A colorful and tangy salad made with kidney beans, garbanzo beans, and green beans, tossed in a sweet and sour dressing. It is a popular summer dish for picnics and barbecues.",
         "pronunciation":"<klas-ik three bin sah-lahd>",
         "original_title":"Classic Three Bean Salad"
      }
   ]
}"""

In [4]:
sample_response_text

'\n{\n   "results":[\n      {\n         "title":"Red Beans and Rice",\n         "cuisine":"Louisiana Creole",\n         "description":"A spicy Cajun dish made with red kidney beans, Andouille sausage, and served over rice. It is a staple dish in New Orleans and is traditionally served on Mondays.",\n         "pronunciation":"<loo-ee-zi-ah-nuh kree-ohl>",\n         "original_title":"Riz Jaune et Haricots Rouge"\n      },\n      {\n         "title":"Frijoles Charros",\n         "cuisine":"Mexican",\n         "description":"A hearty Mexican dish made with kidney beans, bacon, chorizo, and served with a side of corn or flour tortillas. It is a popular dish served at family gatherings and celebrations.",\n         "pronunciation":"<free-hoh-lays charr-ohs>",\n         "original_title":"Frijoles Charros"\n      },\n      {\n         "title":"Greek Kidney Bean Salad",\n         "cuisine":"Greek",\n         "description":"A light and refreshing Greek salad made with kidney beans, feta cheese, 

In [5]:
import json
json.loads(sample_response_text)['results'][0]

{'title': 'Red Beans and Rice',
 'cuisine': 'Louisiana Creole',
 'description': 'A spicy Cajun dish made with red kidney beans, Andouille sausage, and served over rice. It is a staple dish in New Orleans and is traditionally served on Mondays.',
 'pronunciation': '<loo-ee-zi-ah-nuh kree-ohl>',
 'original_title': 'Riz Jaune et Haricots Rouge'}

In [9]:
json_sample = json.loads(sample_response_text)
json_sample['results']


[{'title': 'Red Beans and Rice',
  'cuisine': 'Louisiana Creole',
  'description': 'A spicy Cajun dish made with red kidney beans, Andouille sausage, and served over rice. It is a staple dish in New Orleans and is traditionally served on Mondays.',
  'pronunciation': '<loo-ee-zi-ah-nuh kree-ohl>',
  'original_title': 'Riz Jaune et Haricots Rouge'},
 {'title': 'Frijoles Charros',
  'cuisine': 'Mexican',
  'description': 'A hearty Mexican dish made with kidney beans, bacon, chorizo, and served with a side of corn or flour tortillas. It is a popular dish served at family gatherings and celebrations.',
  'pronunciation': '<free-hoh-lays charr-ohs>',
  'original_title': 'Frijoles Charros'},
 {'title': 'Greek Kidney Bean Salad',
  'cuisine': 'Greek',
  'description': 'A light and refreshing Greek salad made with kidney beans, feta cheese, and a lemon and olive oil dressing. It is a great side dish for any Greek meal or as a delicious vegetarian main course.',
  'pronunciation': '<greek ki-d-

### langchain pipeline test

1. get recos based on input
2. format in json (for easy info retrieval but also storage for later)
3. return results in json output

In [4]:
# 3 components: tool, llm, agent
# llm
llm = OpenAI(openai_api_key = os.getenv("OPENAI_API_KEY"),
            temperature=1.0)
# tool
tools = load_tools(tool_names=['google-search-results-json','serpapi'], llm=llm)
# initialize agent with tools and agent type zero shot
agent = initialize_agent(tools=tools,llm=llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

In [6]:
# get recos
# structure in json format
prompt = """
You are a recipe recommendation engine. Your task is to give at most 5 recipes that involves one or more ingredient given by the user as a JSON format. Return the name of the recipe, type of cuisine if it exists, and a brief description of the dish. If the dish is from a particular cuisine, also include the original language name and english pronunciation of the name. If recipe is not from a specific cuisine, give back 'no specific cuisine' as cuisine type. For description, keep it concise and include things like type of dish, other main ingredients, short description of the dish, and how the dish is typically enjoyed. Format the output in the following JSON schema:
{'results': [
{'title': <recipe title> (original language title and english pronunciation),'cuisine': <cuisine type>,'description':<description>},...
]} 
Do not make up new keys or values.
ingredients: cilantro, lemongrass
"""
agent.run(prompt)



> Entering new AgentExecutor chain...
 I need to find a list of recipes that use cilantro and lemongrass.
Action: Search
Action Input: "recipes with cilantro and lemongrass" 
Observation: Ingredients · 1/2 bunch. fresh cilantro · 1 stalk. lemongrass · 1 (1-inch) piece. fresh ginger · 1 small clove. garlic · 1 tablespoon. rice vinegar.
Thought: I must look for more recipes with cilantro and lemongrass.
Action: Search 
Action Input: "cilantro lemongrass recipes"
Observation: Ingredients · 1/2 bunch. fresh cilantro · 1 stalk. lemongrass · 1 (1-inch) piece. fresh ginger · 1 small clove. garlic · 1 tablespoon. rice vinegar.
Thought: This search is a bit specific, and I should focus more on the total recipe than the ingredients.
Action: Search 
Action Input: "cilantro lemongrass recipes" with individual dish names
Observation: List includes: Ayam goreng, Nasi uduk, Nasi liwet, Sate lilit, ... with chopped lemongrass stalks, shallots, kaffir lime leaves, cilantro, ...
Thought: I can now inc

'{\'results\': [\n{\'title\': \'Ayam Goreng (Malaysian Fried Chicken) | RecipeTin Eats\', \'cuisine\': \'no specific cuisine\', \'description\': \'Fried chicken marinated with shallots, garlic, galangal, turmeric, ginger, and lemongrass. Commonly served with white rice.\'},\n{\'title\': \'Indonesian Fried Chicken - Ayam Goreng - Much Butter\', \'cuisine\': \'no specific cuisine\', \'description\': \'Fried Chicken marinated and cooked with a combination of shallots, garlic, galangal, turmeric, ginger, and lemongrass.\'}, \n{\'title\': "Malaysian Fried Chicken (Ayam Goreng) | Marion\'s Kitchen", \'cuisine\': \'no specific cuisine\', \'description\': \'Fried chicken marinated with fennel, coriander, cumin seeds, red shallots, garlic, and ginger with lemongrass.\'},\n{\'title\': \'Crispy Ayam Goreng Rempah | Malay Spiced Crispy Fried Chicken ...\', \'cuisine\': \'no'

In [8]:
# get recos
# structure in json format
prompt = """
You are a recipe recommendation engine. Your task is to give at most 5 different dish names that involves ingredients given by the user as a JSON format. Return the name of the dish, type of cuisine if it exists, and a brief description of the dish. If the dish is from a particular cuisine, also include the original language name and english pronunciation of the name. If it is not from a specific cuisine, give back 'no specific cuisine' as cuisine type. For description, keep it concise and include things like type of dish, other main ingredients, short description of the dish, and how the dish is typically enjoyed. Format the output in the following JSON schema:
{'results': [
{'title': <recipe title> (original language title and english pronunciation),'cuisine': <cuisine type>,'description':<description>},...
]} 
Do not make up new keys or values.
ingredients: cilantro, lemongrass
"""
agent.run(prompt)



> Entering new AgentExecutor chain...
 What dish recipes could involve cilantro and lemongrass?
Action: Google Search Results JSON
Action Input: "dishes with cilantro and lemongrass"
Observation: [{'title': 'Lemongrass Cilantro Chicken - Rasa Malaysia', 'link': 'https://rasamalaysia.com/lemongrass-cilantro-chicken/', 'snippet': 'Sep 25, 2012 ... A few months ago, when my copy of Food & Wine Annual Cookbook 2012 arrived, the lemongrass cilantro chicken recipe immediately caught my\xa0...'}, {'title': 'Lemongrass-Cilantro Chicken with Honey Dipping Sauce Recipe ...', 'link': 'https://www.foodandwine.com/recipes/lemongrass-cilantro-chicken-honey-dipping-sauce', 'snippet': 'Jun 27, 2019 ... 2 large stalks of fresh lemongrass (cut into 1-inch pieces) · 1 packed cup chopped cilantro · 4 large shallots (coarsely chopped) · ½ cup vegetable\xa0...'}, {'title': 'Lemongrass Cilantro Sauce Recipe | Kitchn', 'link': 'https://www.thekitchn.com/lemongrass-cilantro-sauce-recipe-23185188', 'snippet':

'Agent stopped due to iteration limit or time limit.'

In [24]:
# using sequential chain instead to get 2 chains
# my primary concern is the added cost since ouput of step 1 (reco)
# will be rather large in terms of token...
from langchain.llms import OpenAI
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate

# recipe recommendation chain
llm_reco = OpenAI(temperature=1.0, max_tokens=1000)
reco_template = """Your task is to give at most 5 dish that involves the ingredients given by the user. Return the name of the dish in English translation, original language, english pronunciation. Also return type of cuisine, and a brief description of the dish. For description, keep it concise and include things like type of dish, other main ingredients, short description of the dish, and how the dish is typically enjoyed.
ingredients: {ingredients}"""
reco_prompt_template = PromptTemplate(input_variables=['ingredients'],template=reco_template)
reco_chain = LLMChain(llm=llm_reco,prompt=reco_prompt_template)

# json parsing chain
# not using the output parser for now
llm_parser = OpenAI(temperature=0.0, max_tokens=1000)
parser_template = """Given a list of recipes, format it as a list of JSON documents with the following key value schema:
title: 
    english_translation: <english translation of recipe title>
    original_language: <original language title>
    english_pronunciation: <english pronunciation of original language>
cuisine: <cuisine type>
description': <description>
Do not make up new keys or values. Only use information given in the input list.
input: {recipe_list}"""
parser_prompt_template = PromptTemplate(input_variables=['recipe_list'],template=parser_template)
parser_chain = LLMChain(llm=llm_parser,prompt=parser_prompt_template)


# sequential chain
reco_parsed_result_chain = SimpleSequentialChain(
    chains=[reco_chain,parser_chain],
    verbose=True
)

In [25]:
reco_parsed_result_chain.run("cilantro, lemongrass")



> Entering new SimpleSequentialChain chain...


1. Tom Kha Gai (Thaland): Tom Kha Gai is a soup made with cilantro and lemongrass, coconut milk, and chicken. It is a popular Thai dish that is usually enjoyed with steamed rice.
2. Bún Thịt Nướng (Vietnam): Bún Thịt Nướng is a Vietnamese noodle dish that includes cilantro, lemongrass, pork or other meat, and is topped with peanuts and chives. It is often served with a fish sauce-based dipping sauce.
3. Asian-Style Grilled Chicken (International): This dish combines cilantro and lemongrass with garlic, ginger, and other spices to create a flavorful marinade for chicken. The chicken is grilled or pan-fried until cooked through, and then served with steamed rice, vegetables, or a salad.
4. Pad Thai (Thailand): Pad Thai is a Thai dish that includes cilantro, lemongrass, rice noodles, shrimp, and other ingredients. The noodles are stir-fried with chillies and a sweet and sour sauce and then topped with crushed peanuts, bean sprouts and lime

'\n\nAnswer: \n[\n    {\n        "title": {\n            "english_translation": "Tom Kha Gai",\n            "original_language": "Thaland",\n            "english_pronunciation": ""\n        },\n        "cuisine": "Thai",\n        "description": "Tom Kha Gai is a soup made with cilantro and lemongrass, coconut milk, and chicken. It is a popular Thai dish that is usually enjoyed with steamed rice."\n    },\n    {\n        "title": {\n            "english_translation": "Bún Thịt Nướng",\n            "original_language": "Vietnam",\n            "english_pronunciation": ""\n        },\n        "cuisine": "Vietnamese",\n        "description": "Bún Thịt Nướng is a Vietnamese noodle dish that includes cilantro, lemongrass, pork or other meat, and is topped with peanuts and chives. It is often served with a fish sauce-based dipping sauce."\n    },\n    {\n        "title": {\n            "english_translation": "Asian-Style Grilled Chicken",\n            "original_language": "International",\n   

In [26]:
# using optimized prompt from promptperfect to test'
from langchain.llms import OpenAI
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate

# recipe recommendation chain
llm_reco = OpenAI(temperature=1.0, max_tokens=1000)
reco_template = """Your task is to create 5 different dishes using the ingredients provided by the user. For each dish, please provide the English translation and original language name, as well as the pronunciation of the name in English. In addition, please specify the type of cuisine and provide a brief description of the dish, including its main ingredients and how it is typically enjoyed.
When selecting the ingredients, keep in mind that they must all be used in each dish. The dishes can be from any type of cuisine, including but not limited to Italian, Mexican, Chinese, Indian, or French.
Please ensure that your descriptions are clear and concise, highlighting the most important details about each dish that would be helpful for someone looking to prepare or try the dish. Additionally, your descriptions should be engaging and creative, encouraging the user to want to try each dish.
ingredients: {ingredients}"""
reco_prompt_template = PromptTemplate(input_variables=['ingredients'],template=reco_template)
reco_chain = LLMChain(llm=llm_reco,prompt=reco_prompt_template)

# json parsing chain
# not using the output parser for now
llm_parser = OpenAI(temperature=0.0, max_tokens=1000)
parser_template = """Your task is to format a list of recipes into a list of JSON documents according to the following key-value schema:
"title":
    "english_translation": "English translation of recipe title",
    "original_language": "Original language title",
    "english_pronunciation": "English pronunciation of original language"
"cuisine": "Cuisine type"
"description": "Description"
Please ensure that you use only the information given in the input list and do not create new keys or values. For each recipe, provide an English translation of the recipe title, the original language title, and the English pronunciation of the original language title under the "title" key. Under the "cuisine" key, provide the cuisine type of the recipe, and under the "description" key, provide a description of the recipe.
Your response should be a list of JSON documents, each containing the above key-value schema for a single recipe. Please separate each JSON document with a comma. If necessary, use appropriate data types to represent the values in the JSON document.
Please note that the optimized prompt provides clear and specific instructions on how to format the list of recipes into JSON documents. It also emphasizes the importance of using only the information given in the input list and not creating new keys or values. The optimized prompt also provides a detailed example of the key-value schema, which makes it easier for the user to understand how to structure the response.
input: {recipe_list}"""
parser_prompt_template = PromptTemplate(input_variables=['recipe_list'],template=parser_template)
parser_chain = LLMChain(llm=llm_parser,prompt=parser_prompt_template)


# sequential chain
reco_parsed_result_chain = SimpleSequentialChain(
    chains=[reco_chain,parser_chain],
    verbose=True
)


In [27]:
reco_parsed_result_chain.run("cilantro, lemongrass")



> Entering new SimpleSequentialChain chain...
, avocado, lime, jalapeño 

1. Cilantro-Lemongrass Guacamole - English translation, with the original language of Spanish being “Guacamole de Cilantro y Lemongrass”, pronounced “Gwah-kah-moh-lay day see-lan-troh ee leh-mohn-grass”. This dish is of Mexican cuisine and is one of the lightest and most refreshing types of guacamole. This dish is made by mashing up the avocadoes and mixing with the chopped cilantro and lemongrass, then adding in the jalapeño and the juice of a lime. This guacamole is enjoyed on its own with chips, as a topping for tacos, or to spice up sandwiches. 

2. Lemongrass-Cilantro Sautéed Vegetables - English translation, with the original language of Spanish being “Verduras Salteadas de Cilantro y Lemongrass”, pronounced “Behr-doo-rahs Sah-leh-teh-ah-dahhs day see-lan-troh ee leh-mohn-grass”. This dish is of Chinese cuisine that combines the fresh, citrusy flavors of lemongrass and cilantro with the delicate texture a

'\n\n[\n  {\n    "title": {\n      "english_translation": "Cilantro-Lemongrass Guacamole",\n      "original_language": "Guacamole de Cilantro y Lemongrass",\n      "english_pronunciation": "Gwah-kah-moh-lay day see-lan-troh ee leh-mohn-grass"\n    },\n    "cuisine": "Mexican",\n    "description": "One of the lightest and most refreshing types of guacamole. This dish is made by mashing up the avocadoes and mixing with the chopped cilantro and lemongrass, then adding in the jalapeño and the juice of a lime. This guacamole is enjoyed on its own with chips, as a topping for tacos, or to spice up sandwiches."\n  },\n  {\n    "title": {\n      "english_translation": "Lemongrass-Cilantro Sautéed Vegetables",\n      "original_language": "Verduras Salteadas de Cilantro y Lemongrass",\n      "english_pronunciation": "Behr-doo-rahs Sah-leh-teh-ah-dahhs day see-lan-troh ee leh-mohn-grass"\n    },\n    "cuisine": "Chinese",\n    "description": "Combines the fresh, citrusy flavors of lemongrass and 

In [5]:
# too wordy, I'm not a big fan of the new prompt
# changing it a bit
from langchain.llms import OpenAI
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate

# recipe recommendation chain
llm_reco = OpenAI(temperature=1.0, max_tokens=1000)
reco_template = """Your task is to recommend 5 different dishes using all the ingredients provided by the user. For each dish, please provide the English translation and original language name, as well as the pronunciation of the name in English. In addition, please specify the type of cuisine and provide a brief description of the dish, including its main ingredients and how it is typically enjoyed. Be concise and clear.
ingredients: {ingredients}"""
reco_prompt_template = PromptTemplate(input_variables=['ingredients'],template=reco_template)
reco_chain = LLMChain(llm=llm_reco,prompt=reco_prompt_template)

# json parsing chain
# not using the output parser for now
llm_parser = OpenAI(temperature=0.0, max_tokens=1000)
parser_template = """Your task is to format a list of recipes into a list of JSON documents according to the following key-value schema:
"title":
    "english_translation": "English translation of recipe title",
    "original_language": "Original language title",
    "english_pronunciation": "English pronunciation of original language"
"cuisine": "Cuisine type"
"description": "Description"
Please ensure that you use only the information given in the input list and do not create new keys or values. For each recipe, provide an English translation of the recipe title, the original language title, and the English pronunciation of the original language title under the "title" key. Under the "cuisine" key, provide the cuisine type of the recipe, and under the "description" key, provide a description of the recipe.
input: {recipe_list}"""
parser_prompt_template = PromptTemplate(input_variables=['recipe_list'],template=parser_template)
parser_chain = LLMChain(llm=llm_parser,prompt=parser_prompt_template)


# sequential chain
reco_parsed_result_chain = SimpleSequentialChain(
    chains=[reco_chain,parser_chain],
    verbose=True
)

In [7]:
response = reco_parsed_result_chain.run("cilantro, lemongrass")



> Entering new SimpleSequentialChain chain...
, sesame oil, fish sauce

1. Tom Yar Goong (ต้มยำกุ้ง) – Thai Cuisine – This spicy soup dish features prawns cooked in a flavorful broth with the fresh flavors of cilantro, lemongrass, and sesame oil. It is typically served with fish sauce as a condiment and enjoyed as a main meal.

2. Char Kway Teow (炒粿條) – Cantonese Cuisine – This stir-fried rice noodle dish is a classic in Southeast Asian cuisine. It is made with flat rice noodles, sesame oil, fish sauce, and the fragrant flavors of cilantro and lemongrass for a unique, umami flavor. The dish is typically served as a main course alongside soup or other side dishes.

3. Gai Pad Krapow (ไก่ผัดกระเพรา) – Thai Cuisine – This stir-fried basil dish features diced chicken cooked with the aromatic flavors of cilantro, lemongrass, sesame oil, and fish sauce. It is usually served with steamed or fried rice and topped off with a fried egg, making it a popular, delectable main dish.

4. Laap Plaa 

In [8]:
import json
formatted_response = json.loads(response)
formatted_response

[{'title': {'english_translation': 'Tom Yar Goong',
   'original_language': 'ต้มยำกุ้ง',
   'english_pronunciation': 'tohm yahr goong'},
  'cuisine': 'Thai Cuisine',
  'description': 'This spicy soup dish features prawns cooked in a flavorful broth with the fresh flavors of cilantro, lemongrass, and sesame oil. It is typically served with fish sauce as a condiment and enjoyed as a main meal.'},
 {'title': {'english_translation': 'Char Kway Teow',
   'original_language': '炒粿條',
   'english_pronunciation': 'chahr kway teow'},
  'cuisine': 'Cantonese Cuisine',
  'description': 'This stir-fried rice noodle dish is a classic in Southeast Asian cuisine. It is made with flat rice noodles, sesame oil, fish sauce, and the fragrant flavors of cilantro and lemongrass for a unique, umami flavor. The dish is typically served as a main course alongside soup or other side dishes.'},
 {'title': {'english_translation': 'Gai Pad Krapow',
   'original_language': 'ไก่ผัดกระเพรา',
   'english_pronunciation